# Orchestrate BigQuery and AutoML tables with Kubeflow pipelines

In [1]:
import kfp.components as comp

/home/jupyter/kfp/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
